## Team 14: Multi-Task, Logistic Regression, and Linear Regression Algorithms from Scratch in PySpark
---

#### Summary 

This notebook includes all the code used to run the Linear, Logistic, and Multi-task algorithms from scratch using RDDs in PySpark. For the full toy example implementation, we will train on 2.5% of the data from 2015-2018 and then test on the full 2019 flights dataset. Although our implementation includes both Lasso and Ridge regularization term, for the purposes of illustration we will focus on the basic implementation without any Regularization to compare the results of the Logistic and Linear Regression trained as independent tasks versus the Multi-task model.

_Note: we verified that the from scratch algorithms are working by comparing the From Scratch results on 500 iterations to the MLlib Logistic and Linear regression implementations. We have documented the results in the final report, see sections 5 and 6 for code that we ran._

#### 1. Load Dependencies

In [0]:
# Import Libaries and Create and Connect to spark session 
from pyspark.sql import SparkSession,SQLContext
sql_jar="/path/to/sql_jar_file/sqljdbc42.jar"
spark_snow_jar="/usr/.../snowflake/spark-snowflake_2.11-2.5.5-spark_2.3.jar"
snow_jdbc_jar="/usr/.../snowflake/snowflake-jdbc-3.10.3.jar"
oracle_jar="/usr/path/to/oracle_jar_file//v12/jdbc/lib/oracle6.jar"
spark=(SparkSession
.builder
.master('yarn')
.appName('Spark job new_job')
.config('spark.driver.memory','10g')
.config('spark.submit.deployMode','client')
.config('spark.executor.memory','15g')
.config('spark.executor.cores',4)
.config('spark.yarn.queue','short')
.config('spark.jars','{},{},{},{}'.format(sql_jar,spark_snow_jar,snow_jdbc_jar,oracle_jar))
.enableHiveSupport()
.getOrCreate())

In [0]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,BooleanType,DateType,DoubleType,FloatType
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.rdd import portable_hash
from pyspark.sql.window import Window
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql.functions import substring

import pandas as pd
import numpy as np
import math as math
import time
import datetime
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib.ticker as mtick
import seaborn as sns
from graphframes import *
import geopandas as gpd
import plotly as plotly
!pip install heatmapz
from heatmap import heatmap, corrplot

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 200)


from pyspark.ml import *
from pyspark.ml.linalg import *
from pyspark.ml.stat import *
from pyspark.ml.feature import *
from pyspark.sql.window import *

#Blob credentials
blob_container = "cemgr14c" # The name of your container created in https://portal.azure.com
storage_account = "cemgr14" #The name of your Storage account created in https://portal.azure.com
secret_scope = "w261gr14" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "keygr14" # The name of the secret key created in your local computer using the Databricks CLI
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

Requirement already satisfied: heatmapz in /databricks/python3/lib/python3.8/site-packages (0.0.4)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from heatmapz) (1.2.4)
Requirement already satisfied: matplotlib>=3.0.3 in /databricks/python3/lib/python3.8/site-packages (from heatmapz) (3.4.2)
Requirement already satisfied: seaborn in /databricks/python3/lib/python3.8/site-packages (from heatmapz) (0.11.1)
Requirement already satisfied: kiwisolver>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (1.3.1)
Requirement already satisfied: numpy>=1.16 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (1.19.2)
Requirement already satisfied: cycler>=0.10 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (0.10.0)
Requirement already satisfied: pillow>=6.2.0 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (8.2.0)
Requirement already satisfied: pyparsing>=2.2.1 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (2.4.7)
Requirement already satisfied: python-dateutil>=2.7 in /databricks/python3/lib/python3.8/site-packages (from matplotlib>=3.0.3->heatmapz) (2.8.1)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from cycler>=0.10->matplotlib>=3.0.3->heatmapz) (1.15.0)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.8/site-packages (from pandas->heatmapz) (2020.5)
Requirement already satisfied: scipy>=1.0 in /databricks/python3/lib/python3.8/site-packages (from seaborn->heatmapz) (1.6.2)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

#### 2. Logistic Regression

Logistic Regression is used to solve classification problems, in our case it helps us solve a binary classification problem of delay (1) or no delay (0). In particular the logistic regression the outcome variable (y) is the probability of a flight being delayed (the positive class). Below are the Loss Function and Gradient Descent Update formulas. Unlike the linear regression we cannot use the Mean Squared Error (MSE) as the cost function because the logistic regression (sigmoid function) is non-linear which leads to a non-convex function. Thus to meet the properties of convexity we use the log loss formula in place of MSE.

#### Logistic (Sigmoid) Function

$$g(z)=\frac{1}{1+e^{(-z)}}$$

#### Logistic Regression Loss Function (Log Loss Function)

$$J\left(\theta\right)=-\frac{1}{m}[\sum_{i=1}^{m}{y^{\left(i\right)}log{h_\theta\left(x^{\left(i\right)}\right)}}+\left(1-y^{\left(i\right)}\right)log{\left(1-h_\theta\left(x^{\left(i\right)}\right)\right)}]$$

where ___m___ is the number of examples

Using this loss function above, if y=1 and the prediction=1 then the cost is 0. However, it penalizes heavily if y=1 and the prediction = 0.

#### Gradient Descent Formula
$$
\theta_j=\theta_j-\alpha\sum_{i=1}^{m}{(h_\theta\left(x^{\left(i\right)}\right)}-y^{\left(i\right)})x_j^{\left(i\right)}
$$

where $$h_\theta\left(x\right)=\frac{1}{1+e^{-\theta^Tx}}$$


Notice this formula is similar to the gradient descent update formula for linear regression, but instead here we use Logistic Regression Hypothesis: 
$$h_\theta\left(x\right)=\frac{1}{1+e^{-\theta^Tx}}$$

This section includes the Logistic Regression model implementation from Scratch based on the mathmatical formulas above.

##### - Load Data

In [0]:
def undersampling_Adj(data, column, val1, val2):

    '''
    Input: data = (dataframe with data), column = (column to check ratio for), val1 = majority value for column, val2 = minority value for column
    Output: dataframe with balanced count for minority and majority count by undersampling majority count.
    '''

    major_df = data.filter(col(column) == val1)
    minor_df = data.filter(col(column) == val2)
    n1 = major_df.count()
    n2 = minor_df.count()
    ratio = n2/n1

    sampled_majority_df = major_df.sample(False, ratio, 123)
    combined_df = sampled_majority_df.unionAll(minor_df)

    return combined_df, ratio

In [0]:
# Helper Function to Parse RDD
def parse(line):
    """
    Parse RDD into a tuple
    Args:
        line (line) : line from RDD
    Returns:
        tuple in format (features, (delayLog, delayLN))
    """  
    features, delayLog, delayLN = line[:-2], line[-2], line[-1]
    return (np.array(features, dtype = 'double'), (np.array(delayLog, dtype = 'double'), np.array(delayLN, dtype = 'double')))
  
# Helper function to extract predictions from dataframe from ML model output
def extract(row):
    """
    Get predictions from Spark DataFrame
    Args:
        row
    Returns:
            (LogLabel, probabilities, label, prediction)
    
    """
    return (row.DEP_DEL15,) + tuple(row.probability.toArray().tolist()) +  (row.label,) + (row.prediction,)

We are getting 2.5% of the data for the training data and using all of 2019 for the test dataset. We remove records with missing values and only select relevant numeric features for the model.

In [0]:
# THIS LINK TO UPLOAD THE FULL DATA
# df_airlines = spark.read.parquet(f"{blob_url}/airplanes_weather_final_5yr_EZ/*").cache()

# READ DATA INCLUDING AIRPORT RANK.
airlines = spark.read.parquet(f"{blob_url}/df_airlines_rank_graphs").cache()

# Reduce the amount of data (to run on DBCE)

# CONTINUE WITH A PORTION OF DATA FOR TESTING
proportion = 0.025   # 0.025
(airline_test1, airline_rest) = airlines.randomSplit([proportion, 1- proportion], seed=123)

#CONTINUE WITH ALL DATA
# airline_test = df_airlines

# Select only the columns needed
subset_df = airline_test1.select('YEAR','DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int', 'DEP_DEL15', 'DEP_DELAY').cache()                
subset_df2 =   airlines.select('YEAR','DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int', 'DEP_DEL15', 'DEP_DELAY').cache()  

subset_df2 = subset_df2.na.drop()

# OUTCOME VARIABLES: DEP_TIME  DEP_DEL15
subset_df = subset_df.na.drop()

# Manage unbalanced data
subset_df, rs = undersampling_Adj(subset_df, "DEP_DEL15", 0, 1)

# SPLIT DATA FOR TRAINING AND TESTING
year_train_val = 2018
train = subset_df.filter(subset_df.YEAR <= year_train_val).cache()
test = subset_df2.filter(subset_df2.YEAR > year_train_val).cache()

train = train.drop("YEAR")
test = test.drop("YEAR")

train_rdd = train.rdd
test_rdd = test.rdd

# Parse data as tuple
train_rdd = train_rdd.map(tuple) \
                 .map(parse).cache()

test_rdd = test_rdd.map(tuple) \
                 .map(parse).cache()

# train_rdd.collect()
# test_rdd.collect()

##### - Defining the LogLoss, Sigmoid Functions and Gradient Update

In [0]:
# Based on https://towardsdatascience.com/logistic-regression-from-scratch-69db4f587e17
# helper function: sigmoid function
def sigmoid(z):
    """
    Apply sigmoid function to a value
    Args:
        z - (numeric) numeric value
    Returns:
        value compressed with sigmoid function
    """
    return 1.0/(1+np.exp(-z))

# Log Loss Function
def LogLoss(dataRDD, W):
    """
    Compute the log loss.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][0])).cache()
    #  Log Loss =                                    prediction 1                       +            prediction  0
    # Log Loss =                            y *    log (sigmoid (dot(X,weights)))      + (1-y) * log (1- sigmoid(dot(X,weights)))   . mean
    loss =  augmentedData.map(lambda X_y: ((X_y[1] * np.log(sigmoid(np.dot(X_y[0], W))))+ ((1-X_y[1])*np.log(1-sigmoid(np.dot(X_y[0], W)))))).mean()
    loss = -loss
    return loss

In [0]:
#Function to perform a single GD step
def GDUpdate(dataRDD, W, regType = None, learningRate = 0.1, regParam = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][0])).cache()
    w_broadcast = sc.broadcast(W)
    w = np.append([0.],W[1:])

    if regType =='ridge':
        reg = regParam * 2 * w
    elif regType == 'lasso':
        #reg = 1 * np.sign(w)
        reg = W * 1
#         reg = (reg>0).astype(int) * 2- 1
        reg = (reg>0) * 2- 1
        reg[0] = 0
        reg = reg * regParam
    else:
        reg = np.float(0)
    
    #                               X               .        (sigmoid(X   .    Theta )     - y)          / m 
    grad = augmentedData.map(lambda x: np.dot(x[0], (sigmoid(np.dot(x[0],w_broadcast.value))-x[1]))).mean()
    #
    grad = grad + reg
    new_model = w_broadcast.value - learningRate * grad
  
    #return grad
    return new_model

##### - Defining Function to normalize the data

In [0]:
def normalize(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    normedRDD = dataRDD.map(lambda x: ((x[0]-featureMeans)/featureStdev,x[1])).cache()
    
    return normedRDD

In [0]:
def normalizeLR(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    yMean = refRDD.map(lambda x: x[1][1]).mean()
    yStdev = np.sqrt(refRDD.map(lambda x: x[1][1]).variance())
    
    normedRDD = dataRDD.map(lambda x: ( (x[0]-featureMeans) / featureStdev, (x[1][0], (x[1][1]-yMean)/yStdev))  ).cache()
    
    return normedRDD

##### - Functions to Evaluate Models

In [0]:
def modprediction(data, model):
    """
        Return model predictions for Logistic Regression
        Args:
            data - RDD with records in the following format (features_array, (ylog, ylr))
        Returns:
            predictions - RDD with records are tuples of (prediction, probabilities, label)
    """
    augmentedData = data.map(lambda x: (np.append([1.0], x[0]), x[1][0])).cache()
    
    predictions = augmentedData.map(lambda x: (float(x[1]),sigmoid(np.dot(x[0],model)))) \
                    .map(lambda x: (( float(np.where(x[1]>.5,1,0))   , float(x[0])), x[1]))
    
    return predictions

In [0]:
def rmse(data, model):
    """
        Compute Root Mean Squared Error (RMSE)
        Args:
            data     - each record is a tuple of (features_array, y)
            model  - (array) model coefficients with bias at index 0
    """
    
    augmentedData = data.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()
    
    rmse_result = augmentedData.map(lambda x: (np.dot(x[0],model) - x[1] )**2).mean()
    rmse_result = np.sqrt(rmse_result)
    
    return rmse_result

In [0]:
def logrmetrics(metrics):
    """
        Return evaluation metrics for Logistic Regression model predictions
        Args:
            metrics - SparkDataframe
        Returns:
            dfmetrics - Dataframe of accuracy, recall, precision, f1_score, f05_score, f2_score
    
    """
    dfmetric = {}
    recall = metrics.recall(1.0)
    precision = metrics.precision(1.0)
    dfmetric["accuracy"] = metrics.accuracy
    dfmetric["recall"] = recall
    dfmetric["precision"] = precision
    if (recall + precision) != 0:
        dfmetric["f1_score"] = 2*(recall * precision) / (recall + precision)
    else:
        dfmetric["f1_score"] = "na"
    beta = 0.5
    if ((beta**2 * precision) + recall) != 0:
        dfmetric["f05_score"] = (1+beta**2)*(recall * precision) / ((beta**2 * precision) + recall)
    else:
        dfmetric["f05_score"] = "na"
    beta = 2
    if ((beta**2 *precision) + recall) !=0:
        dfmetric["f2_score"] = (1+beta**2)*(recall * precision) / ((beta**2 *precision) + recall) 
    else:
        dfmetric["f2_score"] = "na"
    
    return dfmetric

In [0]:
def LogRegression(data, nSteps, verbose, model, regType, learningRate, regParam):
    """
         Return model predictions for Logistic Regression
         Args:
              data - RDD with records in the following format (features_array, (ylog, ylr))
              nSteps - (numeric) value for number of steps in gradient descent
              verbose -(boolean) True will print Loss and Models with each step
              model - (numpy array) baseline model to initialize weights
              regType - (str) Regularization type 'Lasso', 'Ridge' or 'none'
              learningRate - (numeric) Learning Rate
              regParam - (numeric) value for amount of regularization         
         Returns:
              model - returns model intercept and weights
    """  
    if verbose: print(f"BASELINE:  Loss = {LogLoss(data,model)}")
    for idx in range(nSteps):
        model = GDUpdate(data, model, regType, learningRate, regParam)
        loss = LogLoss(data, model) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(loss)))
            print("Model:{}".format([w for w in model]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return model

##### - Running Logistic Regression Model

In [0]:
normalizedrdd = normalize(train_rdd, train_rdd).cache()
normalizedtest = normalize(test_rdd, train_rdd).cache()
baseline = np.append(.5,np.zeros(len(train_rdd.take(1)[0][0]))).tolist()
baseline = np.array(baseline)

In [0]:
# Using Model without "Ridge" or "Lasso" regularization
mlogR = LogRegression(normalizedrdd, 500, True, baseline, "none", 0.1, 0.1)
print(mlogR)

BASELINE: Loss = 0.7254721299117749
----------
STEP: 1
Loss: 0.7214695980499971
Model:[0.4874750377334809, -0.0004041513005420325, -0.0003847036389129563, 0.0014160960378463512, 0.004983871212856074, -0.0037013570022236516, -0.0008046823396402761, 0.002658200973594916, 0.001604763218433101, -0.0029364981640192453, 0.0023597391380903673, 0.004688855331887671, -0.0031454343600525758, -0.0006472660266244327, 0.0025467165944713147, 0.001274570867314815, -0.0026516289030124774, 0.001908250255153746, 0.004323059553441408, 0.0014147220271572784, 0.005634587736470192, 0.009034242474910751]
----------
STEP: 2
Loss: 0.7177394406758859
Model:[0.4752461218215004, -0.0008074648953508959, -0.0007582717620889217, 0.0027818607489714965, 0.009750280500007553, -0.0072870171209965945, -0.0015882316967447373, 0.005088427709395046, 0.0030219842191409486, -0.005656494734491504, 0.004632636056662491, 0.009180888661188318, -0.006193350653721192, -0.0012789625837440173, 0.004873022169864358, 0.0023760795432590676, -0.005095594985820502, 0.0037414452639570614, 0.008503732927891226, 0.0027791475000214848, 0.01109834455235712, 0.017760786729280498]
----------
STEP: 3
Loss: 0.7142603384636949
Model:[0.4633093050784179, -0.0012091526309549881, -0.00112105430146067, 0.004099014746778127, 0.014306952632252866, -0.010759723879120053, -0.0023506501881561614, 0.007307779812406678, 0.004267007249073205, -0.008173843991239427, 0.006824011239398428, 0.013483040865141432, -0.009145958953496238, -0.0018949712425114945, 0.006995774077124714, 0.003319334828742324, -0.007345482187155613, 0.005503502345700026, 0.01254831586747243, 0.004094537926578403, 0.016397846758945362, 0.026190512311275706]
----------
STEP: 4
Loss: 0.7110127682371831
Model:[0.45166019387298323, -0.0016084948327721809, -0.0014733953242830457, 0.0053692697477131625, 0.018661522925071772, -0.014122307604534345, -0.003092052597678378, 0.009332418714084195, 0.005354280085589301, -0.010501791257082924, 0.008938987301055343, 0.01760217664580749, -0.01200556212349578, -0.0024952572398600006, 0.008930886050520764, 0.0041182587484832785, -0.009414262176908028, 0.007198197496633894, 0.016462836487080013, 0.005362184077332186, 0.021539516344123565, 0.03433409609460686]
----------
STEP: 5
Loss: 0.7079788788631876
Model:[0.4402940332458825, -0.0020048399571157417, -0.0018156331245470806, 0.006594316687582199, 0.02282150818388197, -0.017377658852390905, -0.0038126520983862624, 0.011177559138852449, 0.006297349667402707, -0.01265294580991648, 0.010982275935712502, 0.021545057471029912, -0.014774529213680483, -0.0030798604005464122, 0.010693321765182423, 0.004785883778966184, -0.01131426744782541, 0.008829123844896194, 0.02025305072778634, 0.006583397826762002, 0.026529587078049106, 0.042201978630038404]
----------
STEP: 6
Loss: 0.7051423609943657
Model:[0.4292057818385115, -0.002397602727402237, -0.0021480996528141917, 0.007775816179470841, 0.026794283044336495, -0.0205287053314965, -0.004512746149281684, 0.012857473773494318, 0.007108870581768326, -0.01463926659759042, 0.012958115716311577, 0.02531831955064024, -0.017455273191678975, -0.0036488861796271812, 0.012297101465924781, 0.005334362586545226, -0.013057178412969378, 0.010399668693393252, 0.023924441925855206, 0.007759504185632992, 0.03137408057204563, 0.04980433716739241]
----------
STEP: 7
Loss: 0.7024883173904173
Model:[0.4183901771966702, -0.002786261273984058, -0.0024711196315608695, 0.008915391059368161, 0.030587061024893624, -0.023578391980578865, -0.005192702874534743, 0.014385508894858632, 0.00780062384824589, -0.016472058651803696, 0.01487029613118161, 0.028928456191492348, -0.020050232260119052, -0.004202496995544005, 0.013755319346018584, 0.005774987801078496, -0.014654021778210492, 0.011913005421895304, 0.027482223648661472, 0.008891833865449596, 0.0360787907141612, 0.05715106489173607]
----------
STEP: 8
Loss: 0.7000031382851948
Model:[0.4078417921419318, -0.003170353705059796, -0.0027850096464604825, 0.010014620756256065, 0.034206879656413, -

In [0]:
# TESTING LOGISTIC REGRESSION
test2 = modprediction(normalizedtest, mlogR)

metrics_logR = MulticlassMetrics(test2.map(lambda x: x[0]))
metrics = logrmetrics(metrics_logR)
dm1 = pd.DataFrame({"Log Reg": list(metrics.values())}, index = list(metrics.keys()))
dm1

Out[72]:

,Log Reg
accuracy,0.587091
recall,0.650730
precision,0.259132
f1_score,0.370660
f05_score,0.294587
f2_score,0.499701


#### 3. Linear Regression

In feature model we will use Linear Regression:

$$
\text{Delay in minutes} = \beta_{0} + \sum_{i=1}^{n}(\beta{i} * \text{Flight}_{i} )+ \sum_{j=1}^{m}(\beta{j} * \text{Weather}_{j} ) + \sum_{k=1}^{l}(\beta{k} * \text{Others}_{k} )
$$


formula for the gradient for linear regression:**  

$$
{\triangledown}f = \frac{2}{n}\sum_{i=1}^{n} (y^i - W_i*X^i) X^i
$$

>
In the formula, we have the error \\(y^i - W_i*X^i\\) for each value as the multiplier (weight) for \\(X^i\\).  As we are adding and dividing over the range of observations (n), this is equal to having the average of the observations \\(X^i\\), multiplied or weighted by the errors \\( y^i\\) - \\(W_i\\)*\\(X^i\\).

>Then the updated weights are calculated as follows:
$$
W_{i+1} = W_i - \alpha * {\triangledown}f
$$

##### - Loss Function and Gradient Update Function

**The loss function for Ridge Regression is:**

$$
f({\theta}) = \frac{1}{n}\sum_{i=1}^{n}[{\theta}_i\cdot{x\text{'}_i} - y_i]^2 + \lambda \sum_{i=1}^{n} {\theta}_i^2
$$

>The gradient for ridge regression is:

$$
{\triangledown}f = \frac{2}{n}\sum_{i=1}^{n} (y^i - W_i*X^i) X^i + 2{\lambda}{W}
$$

>Then the updated weights are calculated as follows:
$$
W_{i+1} = W_i - \alpha * {\triangledown}f
$$

**The loss function for Lasso Regression is:**
$$
f({\theta}) = \frac{1}{n}\sum_{i=1}^{n}[{\beta}_i\cdot{x\text{'}_i} - y_i]^2 + \lambda \sum_{i=1}^{n}|{\beta}_i|
$$

>The gradient for lasso regression is:

$$
{\triangledown}f = \frac{2}{n}\sum_{i=1}^{n} (y^i - W_i*X^i) X^i + {\lambda}  (I_{w>0}(w)*2-1)
$$

>Then the updated weights are calculated as follows:
$$
W_{i+1} = W_i - \alpha * {\triangledown}f
$$

In [0]:
def OLSLoss(dataRDD, W):
    """
    Compute mean squared error.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    #augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()
    loss = None
    
    loss = augmentedData.map(lambda x:(np.dot(x[0],W) - x[1])**2).mean()
    
    return loss

In [0]:

def GDUpdateLR(dataRDD, W, regType = None, learningRate = 0.1, regParam = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()

    w_broadcast = sc.broadcast(W)
    w = np.append([0.],W[1:])

    if regType =='ridge':
        reg = regParam * 2 * w
    elif regType == 'lasso':
        reg = W * 1
        reg = (reg>0) * 2- 1
        reg[0] = 0
        reg = reg * regParam
    else:
        reg = np.float(0)
    
    grad = augmentedData.map(lambda d: np.dot(d[0], ( np.dot(d[0],W) - d[1] )  )).mean()*2
    
    grad = grad + reg
    new_model = W - (learningRate * grad)
   
    return new_model

In [0]:
def OLSRegression(dataLR, nSteps, verbose, modelLR, regType, learningRate, regParam):
    """
         Return model predictions for Logistic Regression
         Args:
              dataLR - RDD with records in the following format (features_array, (ylog, ylr))
              nSteps - (numeric) value for number of steps in gradient descent
              verbose -(boolean) True will print Loss and Models with each step
              modelLR - (numpy array) baseline model to intialize weights
              regType - (str) Regularization type 'Lasso', 'Ridge' or 'none'
              learningRate - (numeric) Learning Rate
              regParam - (numeric) value for amount of regularization         
         Returns:
              modelLR - returns model intercept and weights
    """  
    if verbose: print(f"BASELINE:  Loss = {OLSLoss(dataLR,modelLR)}")
    for idx in range(nSteps):
        modelLR = GDUpdateLR(dataLR, modelLR, regType, learningRate, regParam)
        lossLR = OLSLoss(dataLR, modelLR) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(lossLR)))
            print("Model:{}".format([w for w in modelLR]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return modelLR

##### - Running Linear Regression - Simple, Lasso and Ridge Regulated

In [0]:
%%time
modelLR_none = OLSRegression(normalizedrdd, 500, True, baseline, "none", 0.1, 0.1)
print(modelLR_none)

BASELINE: Loss = 4954.025138527734
----------
STEP: 1
Loss: 4536.889914415618
Model:[6.518600462828188, -0.11859590933414653, -0.12609916433426194, 0.19212956217188362, 0.8177055407360856, -0.7504026873549031, -0.1968831370415015, 0.37242615702018766, 0.45891949444764973, -0.6306089375759147, 0.6048163769786266, 0.8425129421666606, -0.66102903849718, -0.17870601499961494, 0.30937957245136355, 0.38288264920101145, -0.5990258622931471, 0.5447679608311439, 0.9068975614105453, 0.018325839525867575, 2.0423365141060166, 1.1797955839630239]
----------
STEP: 2
Loss: 4283.663120517403
Model:[11.333480833090713, -0.22888657814461372, -0.2201093948671371, 0.30543624065442565, 1.3318849934898709, -1.2851303883747192, -0.34309538160080466, 0.40101100934644096, 0.5866507625374604, -0.8968893476098705, 1.0284239150967731, 1.3929744135810616, -1.1272338509377233, -0.31355967349085345, 0.2895427038930794, 0.46161815147871765, -0.8434259946245207, 0.9236995179888, 1.559766586142525, 0.02253375141287099, 3.6156608142702034, 1.9791656927243815]
----------
STEP: 3
Loss: 4125.855434593314
Model:[15.185385129300728, -0.32134817449032593, -0.2912241157097741, 0.37006354699164623, 1.64884764008239, -1.6613670051176932, -0.4482616212949178, 0.33759361139740307, 0.61811614890001, -1.0100128885148028, 1.3311255685902381, 1.7512210533389816, -1.4500845634414636, -0.411900820030346, 0.18750052100887366, 0.46195391480519293, -0.9408228515903149, 1.1929447242572486, 2.0598091048981, 0.018335117650333025, 4.836729868750802, 2.5458936494747473]
----------
STEP: 4
Loss: 4026.6180754177494
Model:[18.26690856626874, -0.39584355766720325, -0.3453961454819525, 0.4032186817666588, 1.8377238711609856, -1.9243499030771227, -0.5227137607290254, 0.2597157408479788, 0.6261354689873032, -1.0549753728186784, 1.5497536487169352, 1.9826949012950013, -1.6708881991511164, -0.4824681455423389, 0.07770891141968417, 0.45115236927459096, -0.9740669303011772, 1.3864305728376238, 2.4531955795653544, 0.009917458440807703, 5.788820870113311, 2.959625286677032]
----------
STEP: 5
Loss: 3963.9097423695034
Model:[20.732127315843147, -0.45482228533654606, -0.3867932906630622, 0.4157753512845926, 1.9439249162336671, -2.1073628315576385, -0.5749251515754451, 0.18863762775699072, 0.6314359356317178, -1.0687638111054574, 1.70859120774552, 2.130639404963962, -1.8201322356524363, -0.5326563082290419, -0.020589421319232457, 0.44631371151038196, -0.978787364699539, 1.5263278031081033, 2.7658016161066117, -2.246632492445763e-05, 6.533612766221277, 3.268069991651335]
----------
STEP: 6
Loss: 3924.1517522511276
Model:[22.704302315502677, -0.5011202137965411, -0.41846359105084696, 0.4150154246961516, 1.9973794990097553, -2.2341821954432906, -0.6112715660180321, 0.12821713485838315, 0.6387090741348741, -1.0686914950505297, 1.8243802235331295, 2.2237252522657247, -1.9195644130182077, -0.5681246908736067, -0.10487466405149974, 0.4496505173922687, -0.9714190131348077, 1.6278150027412621, 3.0149758314193456, -0.009900041398072046, 7.117717315375255, 3.5019757287854354]
----------
STEP: 7
Loss: 3898.8804078117
Model:[24.282042315230296, -0.5372997715533743, -0.442688384573236, 0.40594085640836464, 2.0177933975061872, -2.3215837996160955, -0.6363865725755333, 0.07747955907939466, 0.648168559230327, -1.0632674291275732, 1.9089517060694223, 2.2809739993965112, -1.9844702089803252, -0.5930387387455623, -0.1770184605611403, 0.4596342081643366, -0.9598990064090386, 1.7015572297769044, 3.213676526287661, -0.01887689455533436, 7.576760290440636, 3.682083402940672]
----------
STEP: 8
Loss: 3882.784131080895
Model:[25.544234315012392, -0.5655000422145257, -0.461196696635764, 0.39200513490701017, 2.0180880142033035, -2.381347248645765, -0.6535866121652828, 0.03451612449295928, 0.6590854196455038, -1.0567140801345722, 1.970781391936818, 2.314992439889717, -2.0255240414070195, -0.6104141390376587, -0.2395368296785828, 0.4742892894100401, -0.9480787673019062, 1.7551581428565128, 3.3720713295689273, -0.026584503937365146

On 2.5 % of Training and Test Data

modelLR_none = [9.59953387514031, -0.2792319760809585, -0.13610874019388292, 0.09113025424615373, 1.02147516098602, -1.4332248187979513, -0.20103124634254874, 0.14475215078420597, 0.1399253081719669, -0.6109345440124277, 2.0489768014831813, 1.2293126151409495, -1.0410810747412278, -0.2223936331208716, -0.3714203250148143, 0.39819825448972956, -0.44132472698144004, 1.1145417744369044, 2.1629760289168947, 0.35206343690682024, 4.480645297739111, 3.064779101029841]

rmse = 48.805181731859925 (test)  
rmse = 41.14886101296988  (train)

In [0]:
print(rmse(normalizedtest, modelLR_none))
print(rmse(normalizedrdd, modelLR_none))

52.296001878094906
62.080181837742

In [0]:
# %%time
# modelLR_lasso = OLSRegression(normalizedrdd, 500, True, baseline, "lasso", 0.1, 0.1)
# print(modelLR_lasso)

On 2.5 % of Training and Test Data

modelLR_lasso = [9.599533875140297, -0.225435266963635, -0.08737547449007968, 0.04836256142201045, 0.9811356848301295, -1.395399157016233, -0.15550883022806483, 0.014510635372070231, 0.15314739867503557, -0.6260577724725453, 2.0103878563781974, 1.197397568363566, -1.0720329129008053, -0.1849426257561813, -0.03321992667839169, 0.17275204054095572, -0.3662288819505662, 1.0847986207757947, 2.12150330147695, 0.3117688343773923, 4.441178495612246, 2.9954111222542754]

rmse = 50.56335989236697  (test)  
rmse = 42.887176843410465  (train)

In [0]:
# print(rmse(normalizedtest, modelLR_lasso))
# print(rmse(normalizedrdd, modelLR_lasso))

In [0]:
# %%time
# modelLR_ridge = OLSRegression(normalizedrdd, 500, True, baseline, "ridge", 0.1, 0.1)
# print(modelLR_ridge)

On 2.5 % of Training and Test Data

modelLR_ridge = [9.599533875140304, -0.2510552936007499, -0.12436994459760073, 0.11460389884298267, 1.0163926328627515, -1.3448453020266176, -0.19537960470796906, 0.1581762091277311, 0.15287614125538387, -0.582538237206741, 1.88966624415537, 1.1769673813410553, -1.0131006810944299, -0.21359370720778978, -0.17683145029249256, 0.2638164979500447, -0.4166130286252367, 1.036594425156843, 1.9670923033243415, 0.33128262742382614, 4.094652409003303, 2.773497418605601]

rmse = 50.54581324445492 (test)  
rmse = 42.87196429524765 (train)

In [0]:
# print(rmse(normalizedtest, modelLR_ridge))
# print(rmse(normalizedrdd, modelLR_ridge))

#### 4. Multi-Task Model - Logistic Regression and Linear Regression

The Multi-Task algorithm combines binary classifcation task (logisic regression) and prediction task (linear regression) by adding their respective weighted loss functions together in a Multi-Task loss function. Below are the formulas for the Multi-Task algorithm. See the final report for additional details and a more comprehensive mathmatical explanations.


___Simplified Multi-Task Loss Function___
$$ Multi-Task\ Loss=\left(\beta\right)LogLoss+\left(1-\beta\right)MSE $$

___Multi-Task Loss Function___

$$ Multi-Task\ Loss =\beta\left(-\frac{1}{m}\sum_{i=1}^{m}{y^{\left(i\right)}log{h_\theta\left(x^{\left(i\right)}\right)}}+\left(1-y\left(i\right)\right)log{\left(1-h_\theta\left(x^{\left(i\right)}\right)\right)}\right)+\left(1-\beta\right)\frac{1}{m}\sum_{i=1}^{m}\left(y^{\left(i\right)}-{\hat{y}}^{\left(i\right)}\right)^2$$
_where __β__=weight of each loss function_

___Multi-Task Gradient___

$$Gradient = \beta\ \ast\ \sum_{i=1}^{m}{{(h}_\theta\left(x^{\left(i\right)})-y^{\left(i\right)}\right)x_j^{\left(i\right)}\ +\ }\left(1-\beta\right)\frac{2}{m}\sum_{i=1}^{m}{{(\left[\theta^T\cdot x_i^\prime-y_i\right]}^\ast x_i^\prime)}$$

##### - Loss and Gradient Update Functions

In [0]:
def MTLoss(data, modelLR, modellogr, beta):
    """
    Compute multi-task loss function
    Args:
        data     - each record is a tuple of (features_array, y)
        modelLR  - (array) Linear Regression model coefficients with bias at index 0
        modellogr - (array) Logistic Regression model coefficients with bias
        beta - (numeric) weight for the loss function of each
    """
    augmentedDF = data.map(lambda x: (np.append([1.0], x[0]), x[1]))
    lossLR = None
    
    multi_loss = augmentedDF.map(lambda x:( (np.dot(x[0],modelLR) - x[1][1])**2, ((x[1][0] * np.log(sigmoid(np.dot(x[0], modellogr))))+ ((1-x[1][0])*np.log(1-sigmoid(np.dot(x[0], modellogr)))))         ))


    
    lossLogR = multi_loss.map(lambda x: x[1]).mean()
    lossLogR = -lossLogR
    lossLR = multi_loss.map(lambda x: x[0]).mean()
    
    # Multi-task loss
    MTLossVal = lossLR*beta + (1-beta)*lossLogR
       
    return MTLossVal

In [0]:
def GDMTUpdateLR(data, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        data - records are tuples of (features_array, y)
        modelLR  - (array) Linear Regression model coefficients with bias at index 0
        modellogr - (array) Logistic Regression model coefficients with bias
        beta - (numeric) weight for the loss function of each
        LRregType - (str) Linear Regression regularization type "lasso" or "ridge" 
        LRregType - (str) Logistic Regression regularization type "lasso" or "ridge"
        LRregParam - (float) Linear Regression regularization value 
        logregParam - (float) Logistic Regression regularization value 
        learningRate = (float) Learning rate value
                
    Returns:
        (new_model_LR, new_model_logR) - (tuple) new models with updated coefficients, bias at index 0
    """

    #helper function 
    def get_reg( W, regType, regParam):
        w = np.append([0.],W[1:])
        if regType =='ridge':
            reg = regParam * 2 * w
        elif regType == 'lasso':
            reg = W * 1
            reg = (reg>0) * 2- 1
            reg[0] = 0
            reg = reg * regParam
        else:
            reg = np.float(0)
        return reg
    
    
    modellogr_broadcast = sc.broadcast(modellogr)
    modelLR_broadcast = sc.broadcast(modelLR)

    #UPDATE MODEL FOR LINEAR REGRESSION
    # add a bias 'feature' of 1 at index 0
    augmentedDF =data.map(lambda x: (np.append([1.0], x[0]), x[1])).cache()
    
    #gradLR = augmentedLR.map(lambda d: np.dot(d[0], ( np.dot(d[0], modelLR) - d[1] )  )).mean()*2
    grads = augmentedDF.map(lambda d: (np.dot(d[0], ( np.dot(d[0], modelLR_broadcast.value) - d[1][1] )  )  ,np.dot(d[0], (sigmoid(np.dot(d[0],modellogr_broadcast.value))-d[1][0])))).cache()
    
    # Get regularization for each model
    lrReg = get_reg(modelLR_broadcast.value, LRregType, LRregParam)
    logReg = get_reg(modellogr_broadcast.value, logregType, logregParam)
    
    # add regularization to the gradients
    gradLR = grads.map(lambda x: x[0]).mean()*2 + lrReg
    grad_LogR = grads.map(lambda x: x[1]).mean() + logReg
    
    new_model_LR = modelLR - (learningRate * gradLR * beta)
        
    new_model_logR = modellogr_broadcast.value - (learningRate * grad_LogR * (1- beta))
    
    return new_model_LR, new_model_logR

In [0]:
def MTRegression(data, nSteps, verbose, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate):

    if verbose: print(f"BASELINE:  Loss = {MTLoss(data, modelLR, modellogr, beta)}")
    for idx in range(nSteps):
        modelLR, modellogr = GDMTUpdateLR(data, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate)
        lossMT = MTLoss(data, modelLR, modellogr, beta) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(lossMT)))
            print("Model LR:{}".format([w for w in modelLR]))
            print("Model LogR:{}".format([w for w in modellogr]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return modelLR, modellogr

##### - Running Multi-Task Linear Regression/Logistic Regression

This is running the From Scratch Multi-Task Linear/Logistic regression. There was no Mllib implementation to compare results to. _Note: Prior to this we validate the separate implementations of the Linear and Logistic Regression models._

In [0]:
normalizedrddLR = normalizeLR(train_rdd, train_rdd).cache()
baselineLR = np.array(np.append(.5,np.zeros(len(train_rdd.take(1)[0][0]))).tolist())
baselineLogR = np.array(np.append(.5,np.zeros(len(train_rdd.take(1)[0][0]))).tolist())
beta = 0.4
LRregType, LRregParam = 'none', 0.1
logregType = "none"
logregParam, learningRate = 0.1, 0.1

modelLR_r, modellogr_r = MTRegression(normalizedrddLR, 500, True, baselineLR, baselineLogR, beta, LRregType, LRregParam, logregType, logregParam, learningRate)

print("************ Linear Regression Model ***********************")
print(modelLR_r)
print("************ Logistic Regression Model ***********************")
print(modellogr_r)

BASELINE: Loss = 0.935283277947065
----------
STEP: 1
Loss: 0.9144439027412792
Model LR:[0.45999999999999996, -0.0007455659110743154, -0.000792735928839578, 0.0012078431108563073, 0.005140593633391566, -0.004717487023047614, -0.0012377269694009933, 0.0023412970027854698, 0.002885046650501205, -0.003964390759464617, 0.003802243059364044, 0.005296548024675566, -0.0041556299884267095, -0.0011234545410183157, 0.0019449478830891168, 0.0024070328630121304, -0.003765840367382943, 0.003424742247194997, 0.0057013088429471335, 0.0001152073568052466, 0.012839367667981952, 0.007416911547602323]
Model LogR:[0.49248502264008853, -0.0002424907803252195, -0.00023082218334777378, 0.0008496576227078107, 0.0029903227277136442, -0.002220814201334191, -0.00048280940378416567, 0.0015949205841569495, 0.0009628579310598605, -0.001761898898411547, 0.0014158434828542203, 0.0028133131991326023, -0.0018872606160315454, -0.0003883596159746596, 0.0015280299566827887, 0.000764742520388889, -0.0015909773418074863, 0.0011449501530922475, 0.0025938357320648446, 0.000848833216294367, 0.003380752641882115, 0.005420545484946451]
----------
STEP: 2
Loss: 0.8969875134833031
Model LR:[0.4231999999999999, -0.001470247107819091, -0.0015047796473007275, 0.0022174748148561987, 0.009517927536858718, -0.00889262846744703, -0.0023480347374803187, 0.0038179559011427945, 0.004937273170500992, -0.007012625364499078, 0.007148810594004674, 0.009858690884469884, -0.00782134661503589, -0.002136636063965487, 0.0030620339717537877, 0.004049244254627465, -0.006639923651630549, 0.00643246515506106, 0.010763826623197472, 0.00019491316674720812, 0.024499337090085865, 0.013877188865214666]
Model LogR:[0.48507637893184524, -0.00048468042297930157, -0.00045764039165644365, 0.0016812243420922506, 0.005902445035682068, -0.004400033790638269, -0.0009580365069995141, 0.0031079068014924784, 0.001858304860770705, -0.003445969079141015, 0.0028003073031914877, 0.005555849171735605, -0.003739465072115723, -0.0007711330289690071, 0.0029768409945903364, 0.00146727998889451, -0.003107301498641958, 0.002262836823207549, 0.005136463741009605, 0.0016795893748850692, 0.006699944731333261, 0.010730465495887659]
----------
STEP: 3
Loss: 0.8822779449427812
Model LR:[0.3893439999999998, -0.0021644639755876846, -0.0021453784269452455, 0.003059215179224209, 0.013238933663790446, -0.012582943171109195, -0.0033406045664608657, 0.004683290813720177, 0.006392977884809394, -0.009357112341881755, 0.01010035876283017, 0.013786849978675965, -0.011048921475241362, -0.0030469274041176795, 0.003599716178912374, 0.005153703823831046, -0.008831088387413292, 0.009079647656803923, 0.015289348128981594, 0.00024486226435216043, 0.03509734910001667, 0.01952948086811895]
Model LogR:[0.4777732586781093, -0.0007263997064313465, -0.0006805292262306323, 0.0024950645531954443, 0.00873801534892508, -0.006538238481111791, -0.0014256740953010416, 0.004542611844237868, 0.002689621163048883, -0.005055168882121304, 0.004154518565836553, 0.008229088681723404, -0.005557078995541743, -0.0011482900654430275, 0.004350035422653078, 0.0021107786561992132, -0.004551873571579774, 0.003354498260859988, 0.007629229416287871, 0.0024925353865078823, 0.009958985492244717, 0.01593207765004323]
----------
STEP: 4
Loss: 0.8698245834131006
Model LR:[0.35819647999999965, -0.002822413568007896, -0.0027224757048270854, 0.003758179438729984, 0.016395585305867422, -0.015841143368159432, -0.004225402181808356, 0.005120488592935343, 0.007423238321717319, -0.011159494266670286, 0.012708100478936484, 0.0171676082848803, -0.013886037112456139, -0.003862274212610666, 0.0037370260476896704, 0.005883715919054655, -0.010497972002166043, 0.01141382739609823, 0.019357526589296313, 0.0002701863692244782, 0.044737374182503065, 0.02449502183815124]
Model LogR:[0.47057478918090945, -0.0009674878578864271, -0.0008995625117358825, 0.003291542553232123, 0.01149867313681382, -0.008636020804111978, -0.001885730174044652, 0.005902574193960768, 0.0034599772649907714, -0.0065923835958

Ordinary Regression: 500 cycles  
Linear Regression Model: [2.59385309251664e-15, -0.0066755325398335195, -0.0032517617727512646, 0.0021781046593111236, 0.024434626665988918, -0.034305982515761925, -0.004809744626119722, 0.003750208219042718, 0.0031371375881927292, -0.014515112316497419, 0.04898028505827534, 0.02937948825565854, -0.024759849917462013, -0.005303913056616505, -0.009381267934277364, 0.009928395977872477, -0.010653979506296344, 0.02661917472825337, 0.05169649103626083, 0.008398731761794863, 0.107083932115518, 0.07329387887352118]  

Logistic Regression Model: [-1.5608584798386809, -0.008930345650416394, -0.00754982060762126, 0.02085226944940337, 0.0809303103085285, -0.12435144920036682, -0.016436544861239383, 0.04196927065542048, -0.016811550220550007, -0.042282945659215414, 0.08411007917968513, 0.10176418844922179, -0.08626204030609436, -0.013895899882409211, 0.027870389931953198, -0.01814050653089008, -0.016875934418088476, 0.06572880574862172, 0.1664458825464685, 0.032862342003531844, 0.1706525313333855, 0.29933201837669987]  

RMSE
50.56727869008469
42.89631028863097

In [0]:
test2 = modprediction(normalizedtest, modellogr_r)
metrics_multi_lr = MulticlassMetrics(test2.map(lambda x: x[0]))
metricsmulti_lr = logrmetrics(metrics_multi_lr )
dm3 = pd.DataFrame({"Log Reg": list(metricsmulti_lr .values())}, index = list(metricsmulti_lr .keys()))
dm3

Out[86]:

,Log Reg
accuracy,0.586973
recall,0.650773
precision,0.259075
f1_score,0.370609
f05_score,0.294530
f2_score,0.499679


In [0]:
print(rmse(normalizedtest, modelLR_r))
print(rmse(normalizedrdd, modelLR_r))

49.82066805033255
70.55708943398905

In [0]:
# LRregType, LRregParam = 'lasso', 0.1
# logregType = "lasso"
# logregParam, learningRate = 0.1, 0.1
# modelLR_lasso, modellogr_lasso = MTRegression(normalizedrddLR, 500, True, baselineLR, baselineLogR, beta, LRregType, LRregParam, logregType, logregParam, learningRate)

# print("************ Linear Regression Model ***********************")
# print(modelLR_lasso)
# print("************ Logistic Regression Model ***********************")
# print(modellogr_lasso)

Linear RegressionL [7.119291095579074e-16, 0.0013753228985011832, 0.0022305590267884157, 0.0005694047593501144, 5.765944750487419e-05, -0.0013846221944906624, -0.003084031779083989, 0.004782094226221991, 0.00241322417068159, -0.0024658133625012507, 0.007667745972616723, 0.005142395505455799, -0.0005382926375558979, 0.0025802628094925436, 0.005291135986926982, 0.003717816609837388, -0.005008949005457102, 0.006522689009120985, 0.004012990915436847, 0.0026599904055640616, 0.0632531478201713, 0.03473985640767136]   

Model LogR:[-1.4926861948541212, 0.0006254816794506101, -0.005202931334545911, -0.0035799266960007195, 0.007445485164502436, 0.0014782303117756024, -0.004181763690276592, -0.0023012786190521398, -0.0046050555703766385, -0.001674735264985538, 0.0018463649283606007, 0.001759927400029589, -1.96714956669378e-05, 0.004453148554301257, 0.000767082804270831, -0.000510357555673145, -0.004146290873988328, -0.0021717090992834642, -0.0009323704292673266, -0.0021064152148614563, -0.0022065918924323736, 0.0019786728487526588]

In [0]:
# test2 = modprediction(normalizedtest, modellogr_lasso)
# metrics_lasso_lr = MulticlassMetrics(test2.map(lambda x: x[0]))
# metricsRlr = logrmetrics(metrics_lasso_lr)
# dm3 = pd.DataFrame({"Log Reg Lasso": list(metricsRlr.values())}, index = list(metricsRlr.keys()))
# dm3

In [0]:
# print(rmse(normalizedtest, modelLR_lasso))
# print(rmse(normalizedrdd, modelLR_lasso))

In [0]:
# LRregType, LRregParam = 'ridge', 0.1
# logregType = "ridge"
# logregParam, learningRate = 0.1, 0.1
# modelLR_ridge, modellogr_ridge = MTRegression(normalizedrddLR, 500, True, baselineLR, baselineLogR, beta, LRregType, LRregParam, logregType, logregParam, learningRate)

# print("************ Linear Regression Model ***********************")
# print(modelLR_ridge)
# print("************ Logistic Regression Model ***********************")
# print(modellogr_ridge)

Model LR:[2.453879181130427e-15, -0.006000400429794389, -0.0029723452591739635, 0.002739054915927632, 0.024293551327300024, -0.03214561113872439, -0.0046699601681354065, 0.0038016331638882524, 0.0036388218720580087, -0.013916284572585737, 0.045163890668018944, 0.028129628280935563, -0.024203710134539123, -0.005104031912128296, -0.004265254197828877, 0.00633715000682583, -0.009965201099429192, 0.024773273070143258, 0.04701374794626212, 0.00791651776996051, 0.09786228720350557, 0.06629068734657798]  

Model LogR:[-1.513568483004315, -0.003212346310817316, -0.003030040825448543, 0.012419592484305167, 0.05821470759191354, -0.05762322691805871, -0.007304049235834673, 0.024032999488324342, 0.004031735320076424, -0.028047678477025172, 0.04719851208767767, 0.0612613612514279, -0.04151958875570505, -0.006092271690781037, 0.019565837028024106, 0.0003644437098173261, -0.019738607401050032, 0.031150061785722598, 0.07352034799055975, 0.019072716715317236, 0.08962246375277483, 0.1366317412321985]

In [0]:
# test2 = modprediction(normalizedtest, modellogr_ridge)
# metrics_lasso_lr = MulticlassMetrics(test2.map(lambda x: x[0]))
# metricsRlr = logrmetrics(metrics_lasso_lr)
# dm3 = pd.DataFrame({"Log Reg Lasso": list(metricsRlr.values())}, index = list(metricsRlr.keys()))
# dm3

In [0]:
# print(rmse(normalizedtest, modelLR_ridge))
# print(rmse(normalizedrdd, modelLR_ridge))

#### 5. PySpark Logistic Regression to Test Results 

Since we have already tested the Lasso and Ridge regressions we are commenting these out. See final report for comparison of results.

##### - Import Dependencies

In [0]:
# ML related libraries
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StandardScaler, Imputer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import LinearRegression

from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml.linalg import Vectors
from itertools import combinations

from pyspark.sql.types import IntegerType

##### - Select Data for Logistic Regression

In [0]:
myY = "DEP_DEL15"

#categoricals = ['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'SEASON', 'WKDAY', 'DEPARTURE_Hour_CRS']

numerics = ['YEAR','DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int']

# 'CRS_DEP_TIME', 'DEP_DELAY', 'DEP_DELAY_NEW', 'DEP_DEL15', 'DEP_DELAY_GROUP'

myX = numerics
print(len(numerics))

# Convert Column to Integer
subset_df = subset_df.withColumn("time_of_day_int", subset_df["time_of_day_int"].cast(IntegerType()))

#CREATE A COPY OF myY with name label to be used in the Grid Search 
# subset_df = subset_df.withColumn("label", subset_df[myY, "DEP_DELAY"])

# SELECT THE COLUMNS WITH THE VARIABLES
dflr = subset_df.select(myX + [myY])

# SELECT THE COLUMNS WITH THE VARIABLES
# dflr = subset_df.select(myX + [myY])


# SPLIT DATA FOR TRAINING AND TESTING
year_train_val = 2018
trainLog = dflr.filter(dflr.YEAR <= year_train_val).cache()
testLog = dflr.filter(dflr.YEAR > year_train_val).cache()

trainLog = trainLog.drop("YEAR")
testLog = testLog.drop("YEAR")

numerics = ['DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int']

myX = numerics

22

##### - Define Imputers and Vector Assembler

In [0]:
# Establish stages for our GBT model
# indexers = map(lambda c: StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid = 'keep'), categoricals)
imputers = Imputer(inputCols = numerics, outputCols = numerics)
featureCols = numerics

# Define vector assemblers
# model_matrix_stages = [imputers] + \
#                      [VectorAssembler(inputCols=featureCols, outputCol="features"), StringIndexer(inputCol="DEP_DEL15", outputCol="label")]

model_matrix_stages = [imputers] + \
                     [VectorAssembler(inputCols=featureCols, outputCol="features")]


# Apply StandardScaler to create scaledFeatures
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True,
                        withMean=True)

In [0]:
# Logistic Regression
def logR_PS(data, maxIter, regParam, elasticNetParam, myY, model_matrix_stages, scaler):
    """
         Return Logistic Regression for MLlib
         Args:
              data - sparkDataframe
              maxIter - (numeric) number of iterations
              regParam - (numeric) value for amount of regularization    
              elasticNetParam - (numeric)
              myY - (str) outcome variable column name
              model_matrix_stages - (list) pre-processing stages
              scaler 
         Returns:
              pipLogR - returns model fitted by LR Pipeline
    """  
    lr = LogisticRegression(maxIter=maxIter, featuresCol = "scaledFeatures", regParam=regParam, elasticNetParam=elasticNetParam, fitIntercept=True, labelCol = myY)

    logrpipeline = Pipeline(stages=model_matrix_stages+[scaler]+[lr])

    pipLogR = logrpipeline.fit(data)
    
    return pipLogR

##### - Ordinary Logistic Regression

In [0]:
# Logist Regression
# Regular: regParam = 0
# Lasso: regParam >0, elasticNetParam = 1
# Ridge: regParam >0, elasticNetParam = 0

maxIter, regParam, elasticNetParam = 100, 0, 0
pipLogR = logR_PS(trainLog, maxIter, regParam, elasticNetParam, myY, model_matrix_stages, scaler)

mod = pipLogR.stages[-1]

print("Coefficients Logistic Regression: ", mod.coefficientMatrix)
print("Intercept Logistic Regression: ", mod.interceptVector)

logR_pred = pipLogR.transform(testLog)

# preddf = logR_pred.rdd.map(extract).toDF(["DEP_DEL15", "p0", "p1", "label", "prediction"])
preddf = logR_pred.select("prediction","DEP_DEL15").rdd
# Get performance metrics
metricsLR = MulticlassMetrics(preddf)
mLR = logrmetrics(metricsLR)
dmPS1 = pd.DataFrame({"LogRegPS": list(mLR.values())}, index = list(mLR.keys()))
dmPS1

Coefficients Logistic Regression: DenseMatrix([[-0.01865246, -0.01546011, 0.03898819, 0.08064467, -0.12331954,
 -0.04025841, 0.04268439, -0.03224815, -0.03438428, 0.10680683,
 0.0950519 , -0.10823849, -0.02903456, 0.05051912, -0.03489512,
 -0.01616437, 0.06814149, 0.17242387, 0.03413295, 0.26457321,
 0.30648593]])
Intercept Logistic Regression: [-0.004795843318296452]
Out[98]:

,LogRegPS
accuracy,0.611427
recall,0.651446
precision,0.608865
f1_score,0.629436
f05_score,0.616930
f2_score,0.642460


##### - Ridge Logistic Regression

In [0]:
# Logist Regression
# Regular: regParam = 0
# Lasso: regParam >0, elasticNetParam = 1
# Ridge: regParam >0, elasticNetParam = 0

# maxIter, regParam, elasticNetParam = 100, 0.1, 0
# pipLogR = logR_PS(trainLog, maxIter, regParam, elasticNetParam, myY, model_matrix_stages, scaler)

# mod = pipLogR.stages[-1]

# print("Coefficients Logistic Regression: ", mod.coefficientMatrix)
# print("Intercept Logistic Regression: ", mod.interceptVector)

# logR_pred = pipLogR.transform(testLog)

# # preddf = logR_pred.rdd.map(extract).toDF(["DEP_DEL15", "p0", "p1", "label", "prediction"])
# preddf = logR_pred.select("prediction","DEP_DEL15").rdd
# # Get performance metrics
# metricsLR = MulticlassMetrics(preddf)
# mLR = logrmetrics(metricsLR)
# dmPS1 = pd.DataFrame({"LogReg-Ridge": list(mLR.values())}, index = list(mLR.keys()))
# dmPS1

##### - Lasso Logistic Regression

In [0]:
# Logist Regression
# Regular: regParam = 0
# Lasso: regParam >0, elasticNetParam = 1
# Ridge: regParam >0, elasticNetParam = 0

# maxIter, regParam, elasticNetParam = 100, 0.1, 0
# pipLogR = logR_PS(trainLog, maxIter, regParam, elasticNetParam, myY, model_matrix_stages, scaler)

# mod = pipLogR.stages[-1]

# print("Coefficients Logistic Regression: ", mod.coefficientMatrix)
# print("Intercept Logistic Regression: ", mod.interceptVector)

# logR_pred = pipLogR.transform(testLog)

# # preddf = logR_pred.rdd.map(extract).toDF(["DEP_DEL15", "p0", "p1", "label", "prediction"])
# preddf = logR_pred.select("prediction","DEP_DEL15").rdd
# # Get performance metrics
# metricsLR = MulticlassMetrics(preddf)
# mLR = logrmetrics(metricsLR)
# dmPS1 = pd.DataFrame({"LogRegPS_Lasso": list(mLR.values())}, index = list(mLR.keys()))
# dmPS1

#### 6. PySpark Linear Regression

Since we have already tested the Lasso and Ridge regressions we are commenting these out. See final report for comparison of results.

Reference: https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a

In [0]:
def rmsePS(data):
    """
    Compute Root Mean Squared Error (RMSE)
    Args:
        data     - each record is a tuple of (features_array, y)
        model  - (array) model coefficients with bias at index 0
    """
        
    pred_lab = data.select("prediction", "DEP_DELAY").rdd.cache()

    rmse_result = pred_lab.map(lambda x: (x[0] - x[1] )**2).mean()
    rmse_result = np.sqrt(rmse_result)

    return rmse_result

##### - Prepare Data for Linear Regression

In [0]:
myYLR = "DEP_DELAY"

numerics = ['YEAR','DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int']

# SELECT THE COLUMNS WITH THE VARIABLES
dfLN = subset_df.select(numerics + [myYLR])

# SELECT THE COLUMNS WITH THE VARIABLES
# dflr = subset_df.select(myX + [myY])


# SPLIT DATA FOR TRAINING AND TESTING
year_train_val = 2018
trainLN = dfLN.filter(dfLN.YEAR <= year_train_val).cache()
testLN = dfLN.filter(dfLN.YEAR > year_train_val).cache()

trainLN = trainLN.drop("YEAR")
testLN = testLN.drop("YEAR")

numerics = ['DAY_OF_MONTH','DAY_OF_WEEK', 'DISTANCE','wind_speed_mps_orig','ceiling_ht_dim_orig','visibility_meters_orig','temp_cels_orig',
                                'dew_pt_orig','atmos_press_orig','precip_milimeters_orig', 'wind_speed_mps_dest','ceiling_ht_dim_dest','visibility_meters_dest',
                                'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest','precip_milimeters_dest', 'rolling_ninety_day_average','Air_Page_Rank_traffic',
                                'OD_delay_pair', 'time_of_day_int']

myX = numerics

In [0]:
trainLN.select(mean("DEP_DELAY")).collect()

Out[103]: [Row(avg(DEP_DELAY)=30.593002314140932)]

##### - Imputers and Vector Assembler

This is used for the PySpark MLlib implementation that we used to verify that our from scratch implementation is working.

In [0]:
# indexers = map(lambda c: StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid = 'keep'), categoricals)
imputers = Imputer(inputCols = numerics, outputCols = numerics)
featureCols = numerics

# Define vector assemblers
# model_matrix_stagesLR = [imputers] + \
#                      [VectorAssembler(inputCols=featureCols, outputCol="features"), StringIndexer(inputCol=myYLR, outputCol="label")]

model_matrix_stagesLR = [imputers] + \
                     [VectorAssembler(inputCols=featureCols, outputCol="features")]


# Apply StandardScaler to create scaledFeatures
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True,
                        withMean=True)

##### - Ordinary Linear Regression

Below is the Linear Regression MLlib implementation without any Regularization applied.

In [0]:
# LINEAR REGRESSION
# Regular: regParam = 0
# Lasso: regParam >0, elasticNetParam = 1
# Ridge: regParam >0, elasticNetParam = 0
regParam = 0
elasticNetParam = 0

lnR = LinearRegression(maxIter=200, featuresCol = "scaledFeatures", labelCol = myYLR, regParam=regParam, elasticNetParam = elasticNetParam, fitIntercept=True)

lrpipeline = Pipeline(stages=model_matrix_stagesLR+[scaler]+[lnR])

pipLR = lrpipeline.fit(trainLN)

modLR = pipLR.stages[-1]

print("Coefficients Linear Regression: ", modLR.coefficients)
print("Intercept Linear Regression: ", modLR.intercept)

LR_pred = pipLR.transform(testLN)

print("RMSE: ",rmsePS(LR_pred))

Coefficients Linear Regression: [-0.668510957268133,-0.5092408169967343,0.20845196219783957,1.7832934713600568,-2.3790594918716277,-0.6740870825511257,-0.48834629278389297,1.017557821371781,-1.1138005353859635,2.1172232485722695,2.340165659008655,-1.8126971238832383,-0.6273422725589477,-1.3335809486543322,1.2989536542443276,-1.0060385617109326,1.8561382089377578,3.9856509043857358,-0.052422187611202334,9.292819021939586,4.658951411678922]
Intercept Linear Regression: 30.593002314140822
RMSE: 71.47839578705111

##### - Lasso Linear Regression

In [0]:
# # LINEAR REGRESSION
# # Regular: regParam = 0
# # Lasso: regParam >0, elasticNetParam = 1
# # Ridge: regParam >0, elasticNetParam = 0
# regParam = 0.1
# elasticNetParam = 1

# lnR = LinearRegression(maxIter=200, featuresCol = "scaledFeatures", labelCol = myYLR, regParam=regParam, elasticNetParam = elasticNetParam, fitIntercept=True)

# lrpipeline = Pipeline(stages=model_matrix_stagesLR+[scaler]+[lnR])

# pipLR = lrpipeline.fit(trainLN)

# modLR = pipLR.stages[-1]

# print("Coefficients Linear Regression: ", modLR.coefficients)
# print("Intercept Linear Regression: ", modLR.intercept)

# LR_pred = pipLR.transform(testLN)

# print("RMSE: ",rmsePS(LR_pred))

##### - Ridge Linear Regression

In [0]:
# # LINEAR REGRESSION
# # Regular: regParam = 0
# # Lasso: regParam >0, elasticNetParam = 1
# # Ridge: regParam >0, elasticNetParam = 0
# regParam = 0.1
# elasticNetParam = 0

# lnR = LinearRegression(maxIter=200, featuresCol = "scaledFeatures", labelCol = myYLR, regParam=regParam, elasticNetParam = elasticNetParam, fitIntercept=True)

# lrpipeline = Pipeline(stages=model_matrix_stagesLR+[scaler]+[lnR])

# pipLR = lrpipeline.fit(trainLN)

# modLR = pipLR.stages[-1]

# print("Coefficients Linear Regression: ", modLR.coefficients)
# print("Intercept Linear Regression: ", modLR.intercept)

# LR_pred = pipLR.transform(testLN)

# print("RMSE: ",rmsePS(LR_pred))